In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Avocado Prices in the United States (2015-18)

Analysing the prices by state and type, and price changes throughout the period and training a regression model, ready to predict the forecast for the future of avocado prices.


# Loading the data

In [5]:
# Loading the dataset
avocado_filepath = '/content/drive/MyDrive/Prak Data Mining/avocado.csv'
df = pd.read_csv(avocado_filepath, parse_dates=['Date'])

In [6]:
df.head()

,Unnamed: 0,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,0,2015-12-27,1.33,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.0,conventional,2015,Albany
1,1,2015-12-20,1.35,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,0.0,conventional,2015,Albany
2,2,2015-12-13,0.93,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.0,conventional,2015,Albany
3,3,2015-12-06,1.08,78992.15,1132.00,71976.41,72.58,5811.16,5677.40,133.76,0.0,conventional,2015,Albany
4,4,2015-11-29,1.28,51039.60,941.48,43838.39,75.78,6183.95,5986.26,197.69,0.0,conventional,2015,Albany


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18249 entries, 0 to 18248
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Unnamed: 0    18249 non-null  int64         
 1   Date          18249 non-null  datetime64[ns]
 2   AveragePrice  18249 non-null  float64       
 3   Total Volume  18249 non-null  float64       
 4   4046          18249 non-null  float64       
 5   4225          18249 non-null  float64       
 6   4770          18249 non-null  float64       
 7   Total Bags    18249 non-null  float64       
 8   Small Bags    18249 non-null  float64       
 9   Large Bags    18249 non-null  float64       
 10  XLarge Bags   18249 non-null  float64       
 11  type          18249 non-null  object        
 12  year          18249 non-null  int64         
 13  region        18249 non-null  object        
dtypes: datetime64[ns](1), float64(9), int64(2), object(2)
memory usage: 1.9+ MB


# Preprocessing the data

In [8]:
df = df.rename(columns={'4046':'Small/Med Hass', '4225':'Large Hass', '4770':'Extra Large Hass'})

In [9]:
df.columns

Index(['Unnamed: 0', 'Date', 'AveragePrice', 'Total Volume', 'Small/Med Hass',
       'Large Hass', 'Extra Large Hass', 'Total Bags', 'Small Bags',
       'Large Bags', 'XLarge Bags', 'type', 'year', 'region'],
      dtype='object')

In [10]:
df = df.drop('Unnamed: 0', axis=1)

In [11]:
df.isnull().any()

Date                False
AveragePrice        False
Total Volume        False
Small/Med Hass      False
Large Hass          False
Extra Large Hass    False
Total Bags          False
Small Bags          False
Large Bags          False
XLarge Bags         False
type                False
year                False
region              False
dtype: bool

In [12]:
print(df['type'].unique(), "\n")
print(df['type'].value_counts())

['conventional' 'organic'] 

conventional    9126
organic         9123
Name: type, dtype: int64


In [13]:
df['region'].unique()

array(['Albany', 'Atlanta', 'BaltimoreWashington', 'Boise', 'Boston',
       'BuffaloRochester', 'California', 'Charlotte', 'Chicago',
       'CincinnatiDayton', 'Columbus', 'DallasFtWorth', 'Denver',
       'Detroit', 'GrandRapids', 'GreatLakes', 'HarrisburgScranton',
       'HartfordSpringfield', 'Houston', 'Indianapolis', 'Jacksonville',
       'LasVegas', 'LosAngeles', 'Louisville', 'MiamiFtLauderdale',
       'Midsouth', 'Nashville', 'NewOrleansMobile', 'NewYork',
       'Northeast', 'NorthernNewEngland', 'Orlando', 'Philadelphia',
       'PhoenixTucson', 'Pittsburgh', 'Plains', 'Portland',
       'RaleighGreensboro', 'RichmondNorfolk', 'Roanoke', 'Sacramento',
       'SanDiego', 'SanFrancisco', 'Seattle', 'SouthCarolina',
       'SouthCentral', 'Southeast', 'Spokane', 'StLouis', 'Syracuse',
       'Tampa', 'TotalUS', 'West', 'WestTexNewMexico'], dtype=object)

# Exploratory Data Analysis (EDA)

In [14]:
import plotly.express as px

fig = px.histogram(df, x='AveragePrice', color='type', marginal='rug', hover_data=df.columns)
fig.show()

Output hidden; open in https://colab.research.google.com to view.

In [15]:
fig = px.bar(df, x='AveragePrice', y='region', color='type', hover_data=df.columns, height=1000)
fig.show()

Output hidden; open in https://colab.research.google.com to view.

In [16]:
df_date_sorted = df.sort_values(by='Date')

df3 = df_date_sorted.loc[df_date_sorted.type == 'conventional']
df4 = df_date_sorted.loc[df_date_sorted.type == 'organic']

In [17]:
df3_alt = df3.groupby('Date').sum()
df4_alt = df4.groupby('Date').sum()

In [18]:
fig = px.line(df3_alt, x=df3_alt.index, y='Total Bags',title=
              'Sales of conventional avocados across the United States (2015-2018)')
fig.show()

In [19]:
fig = px.line(df4_alt, x=df4_alt.index, y='Total Bags', title=
              'Sales of organic avocados across the United States (2015-2018)')
fig.show()

In [20]:
df_sf = df_date_sorted.loc[df_date_sorted.region == 'SanFrancisco']
df_htsprng = df_date_sorted.loc[df_date_sorted.region == 'HartfordSpringfield']
df_ny = df_date_sorted.loc[df_date_sorted.region == 'NewYork']

df_sf = df_sf.groupby('Date').sum()
df_htsprng = df_htsprng.groupby('Date').sum()
df_ny = df_ny.groupby('Date').sum()

In [21]:
fig = px.line(df_sf, x=df_sf.index, y='Total Bags', title=
              'Total sales of avocados in San Francisco, CA, US (2015-2018)')
fig.show()

In [22]:
fig = px.line(df_htsprng, x=df_htsprng.index, y='Total Bags', title=
              'Total sales of avocados in the Hartford-Springfield area, CT/MA, US (2015-2018)')
fig.show()

In [23]:
fig = px.line(df_ny, x=df_ny.index, y='Total Bags', title=
              'Total sales of avocados in New York, US (2015-2018)')
fig.show()

In [24]:
fig = px.imshow(df.corr())
fig.show()

# Feature selection and engineering

In [25]:
df_copy = df.copy()

df1 = df_copy.drop(['Date', 'AveragePrice'], axis=1)
df2 = df_copy.pop('AveragePrice')

for colname in df1.select_dtypes('object'):
    df1[colname], _ = df1[colname].factorize()
    
discrete_features = df1.dtypes == int
print(discrete_features)

Total Volume        False
Small/Med Hass      False
Large Hass          False
Extra Large Hass    False
Total Bags          False
Small Bags          False
Large Bags          False
XLarge Bags         False
type                 True
year                 True
region               True
dtype: bool


In [26]:
from sklearn.feature_selection import mutual_info_regression

def make_mi_scores(X, y, discrete_features):
    mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features)
    mi_scores = pd.Series(mi_scores, name='MI Scores', index=df1.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

mi_scores = make_mi_scores(df1, df2, discrete_features)
mi_scores

Total Volume        0.321990
Small/Med Hass      0.298386
Total Bags          0.289077
type                0.254651
region              0.253691
Large Hass          0.252122
Small Bags          0.237934
Extra Large Hass    0.222763
Large Bags          0.196360
XLarge Bags         0.121440
year                0.047780
Name: MI Scores, dtype: float64

In [27]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

label_encoder = LabelEncoder()
df1['type'] = label_encoder.fit_transform(df1['type'])

OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
df_ohe = pd.get_dummies(data=df1, columns=['region'])


In [28]:
df_ohe

,Total Volume,Small/Med Hass,Large Hass,Extra Large Hass,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,...,region_44,region_45,region_46,region_47,region_48,region_49,region_50,region_51,region_52,region_53
0,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.0,0,2015,...,0,0,0,0,0,0,0,0,0,0
1,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,0.0,0,2015,...,0,0,0,0,0,0,0,0,0,0
2,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.0,0,2015,...,0,0,0,0,0,0,0,0,0,0
3,78992.15,1132.00,71976.41,72.58,5811.16,5677.40,133.76,0.0,0,2015,...,0,0,0,0,0,0,0,0,0,0
4,51039.60,941.48,43838.39,75.78,6183.95,5986.26,197.69,0.0,0,2015,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18244,17074.83,2046.96,1529.20,0.00,13498.67,13066.82,431.85,0.0,1,2018,...,0,0,0,0,0,0,0,0,0,1
18245,13888.04,1191.70,3431.50,0.00,9264.84,8940.04,324.80,0.0,1,2018,...,0,0,0,0,0,0,0,0,0,1
18246,13766.76,1191.92,2452.79,727.94,9394.11,9351.80,42.31,0.0,1,2018,...,0,0,0,0,0,0,0,0,0,1
18247,16205.22,1527.63,2981.04,727.01,10969.54,10919.54,50.00,0.0,1,2018,...,0,0,0,0,0,0,0,0,0,1


In [29]:
df2

0        1.33
1        1.35
2        0.93
3        1.08
4        1.28
         ... 
18244    1.63
18245    1.71
18246    1.87
18247    1.93
18248    1.62
Name: AveragePrice, Length: 18249, dtype: float64

# Training the model

In [30]:
feature_names = ['Total Volume',
                'Total Bags',
                'type',
                'region']

columns_to_drop = ['Small/Med Hass',
                  'Large Hass',
                   'Extra Large Hass',
                  'Small Bags',
                  'Large Bags',
                  'XLarge Bags']

X = df_ohe.drop(columns_to_drop, axis=1)
y = df2

In [31]:
from sklearn.model_selection import train_test_split

train_X, val_X, train_y, val_y = train_test_split(X, y, random_state = 0)

In [32]:
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.tree import DecisionTreeRegressor

def get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=0)
    model.fit(train_X, train_y)
    preds_val = model.predict(val_X)
    mae = mean_absolute_error(val_y, preds_val)
    return(mae)

for max_leaf_nodes in [5, 50, 500, 5000]:
    my_mae = get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y)
    print("Max leaf nodes: {}  \t\t Mean Absolute Error:  {}".format(max_leaf_nodes, my_mae))

Max leaf nodes: 5  		 Mean Absolute Error:  0.23121812446229142
Max leaf nodes: 50  		 Mean Absolute Error:  0.1978591118639939
Max leaf nodes: 500  		 Mean Absolute Error:  0.14792011818422526
Max leaf nodes: 5000  		 Mean Absolute Error:  0.14502225364842175


In [33]:
avocado_model = DecisionTreeRegressor(max_leaf_nodes=500, random_state=1)

avocado_model.fit(train_X, train_y)

DecisionTreeRegressor(max_leaf_nodes=500, random_state=1)

In [34]:
print("Prediction for the first five entries:")
display(val_X[:5])
print("The predictions are:")
print(avocado_model.predict(val_X))

Prediction for the first five entries:


,Total Volume,Total Bags,type,year,region_0,region_1,region_2,region_3,region_4,region_5,...,region_44,region_45,region_46,region_47,region_48,region_49,region_50,region_51,region_52,region_53
9181,4400.25,1305.74,1,2015,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1013,190716.43,52872.97,0,2015,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14625,1045450.41,578257.49,1,2016,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
15234,9883.59,5339.26,1,2017,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18247,16205.22,10969.54,1,2018,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


The predictions are:
[1.78257353 1.17346457 1.42491228 ... 2.08928571 1.49151625 1.59654545]


In [35]:
# Out of sample
predicted_avocado_prices = avocado_model.predict(X)
mean_absolute_error(y, predicted_avocado_prices)

0.12734001747463083

In [36]:
# In sample
val_avocado_prices = avocado_model.predict(val_X)
print("DecisionTreeRegressor")
print("Mean Absolute Error: {}".format(mean_absolute_error(val_y, val_avocado_prices)))
print("r2_score: {}".format(r2_score(val_y, val_avocado_prices)))

DecisionTreeRegressor
Mean Absolute Error: 0.1474682685934868
r2_score: 0.74611504012754


In [37]:
from sklearn.ensemble import RandomForestRegressor

forest_model = RandomForestRegressor(random_state=1)
forest_model.fit(train_X, train_y)
avocado_preds = forest_model.predict(val_X)
print("RandomForestRegressor")
print("r2_score: {}".format(r2_score(val_y, avocado_preds)))
print("Mean Absolute Error: {}".format(mean_absolute_error(val_y, avocado_preds)))

RandomForestRegressor
r2_score: 0.8192816609047016
Mean Absolute Error: 0.11829048871356561


In [38]:
from xgboost import XGBRegressor

xgb_model = XGBRegressor()
xgb_model.fit(train_X, train_y)

xgb_preds = xgb_model.predict(val_X)

print("XGBoost")
print("r2_score: {}".format(r2_score(val_y, xgb_preds)))
print("Mean Absolute Error: {}".format(mean_absolute_error(xgb_preds, val_y)))


[08:15:58] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
XGBoost
r2_score: 0.6324106376522847
Mean Absolute Error: 0.18092009750657143


In [39]:
scores = -1 * cross_val_score(forest_model, X, y,
                             cv=5,
                             scoring='neg_mean_absolute_error')

print('MAE scores:', scores)

MAE scores: [0.14800164 0.19991164 0.21695989 0.27408052 0.33028186]


In [40]:
print("Average MAE score (across experiments):")
print(scores.mean())

Average MAE score (across experiments):
0.2338471113346873
